<center> <h1>Developing a data pipeline APIs to Cloud with Python</h1> </center>
<center><img src = "https://cdn.pixabay.com/photo/2018/11/28/10/45/cloud-3843352_960_720.jpg" style="width:700px;height:400px;"></img></center>

<center><h2>The Project's Objectives and  Environment</h2></center>
<p style="text-align: justify">The objective of the project was to develope my skills in creating a phyton based datapipeline. Various APIs were to be used to 
to retrieve data and store them in a structured way (database) in a cloud. The data within the database should be updated if appropriate.
Based on these structured and up to date informations I should be able to make profound and valuable recommendations to a company called "Gans". The company does rent scooters
in various European Cities. To this end I virtually became a Data Scientist at Gans with the task to obtain publically available information that could help
the company to strategically and "data-based" distribute their scooters in the cities, make intelligent assumptions were to expect large numbers of left scooters and
where and when a high scooter-demand will occurr. The buisness of scooter rental is a comparatively young service that is in large parts digitized. Both facts, the 
relatively little extent of experience in the dynamic timely and spacially distribution of the scooter-demand and the high degree of digitalization of the service make it an ideal
place for data-based decision development in the service. The directs steps in which these objectives were suppossed to be acchieved were:
<ul style="margin: 20px"><li style="margin: 20px">1. <strong>Data Retrieval and Collection:</strong> Write phyton request to usefull API's and store the data in clearly structured dataframes locally.</li>
<li style="margin: 20px"><span style="color: #d13a11"><strong>2. Data Transfer to a Cloud:</strong></span> Transfer the data to a professional cloud service. Use a serverless offer(see below).</li>
<li style="margin: 20px">3. <strong>Data Updating in the Cloud:</strong> Decide and implement which part of the database should be fed with repeated updates.</li></ul></p>

<h3>Data Retrival and Collection</h3>
<p style="text-align: justify">For the first objective I selected four APIs for data retrival. 1. GeoDB for obtaining basic data on the
cities were the scooters are rented. For example the population if the city will be important for decisions about the extent of the service provided by Gans.
The GeoDB API demands a "wikidata id" of the city which needed to be obtained by web-scrapping using the beautifull soup package of python. The following code was used to obtain the wikidata-id.
\d+ is a regular expression and means one digit or more, the wiki data id consist of a Q followed by severaldigits</p>

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import re

a = ['Berlin', 'Paris']

def idScrapping(cities):
    cities_id = [] # initiate an empty id list
    for city in cities:
        #retrieve the wikidataId
        time.sleep(1)
        url1 = 'https://en.wikipedia.org/wiki/{}'.format(city) 
        citem = requests.get(url1, 'html.parser') 
        if BeautifulSoup(citem.content) != None:
            soup = BeautifulSoup(citem.content)
        if soup.find('li', {'id':'t-wikibase'}).find('a')['href'] != None:
            wikidata_link = soup.find('li', {'id':'t-wikibase'}).find('a')['href'] 
        city_id = re.search('Q\d+', wikidata_link).group()
        cities_id.append(city_id)
      
    return cities_id

idScrapping(a)

In [ ]:
<p style="text-align: justify">The 'beautifull soup' python library simplifies the finding of information in a website`s html- or xml-code. The wiki-data-id consist of a 'Q' followed by several digits. Here I used a regular expression to retrieve this were:
'\d+' means one digit or more. The return is a list of the cities wiki-data-id. Using this list the geoDB API can be used to retrieve data about the cities. A simplified way how I did this is shown here:  </p>

In [ ]:
def cities_info(cities):
    dfList = []
    for city in cities:
        url2 = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/{}".format(city_id)
        headers = {
        "X-RapidAPI-Key": API_key,
        "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
        }    
        response = requests.request("GET", url2, headers=headers)
        print(response.json())
        cit_dic = {}#make a dictionary to retrieve the information
        cit_dic['City'] = response.json()['data']['name']
        cit_dic['Country'] = response.json()['data']['country']
        cit_dic['CountryCode'] = response.json()['data']['countryCode']
        cit_dic['WikiDataId'] = response.json()['data']['wikiDataId']
        cit_dic['Latitude'] = round(response.json()['data']['latitude'], 4)
        cit_dic['Longitude'] = round(response.json()['data']['longitude'], 4)
        cit_dic['Population'] = response.json()['data']['population']
        cit_dic['Timezone'] = response.json()['data']['timezone']
        
        dfList.append(cit_dic) #put it in a list
        
    df_demo = pd.DataFrame(dfList) # transform the list to df
    df_demo.to_csv('./csvTables/City.csv') #writes the df to csv to the folder csv tables and replaces the table of the same name that might be already there
    return df_demo

demo(citd) 

In [ ]:
<p style="text-align: justify">The code needs the list 'city_id'. The items from the list are appended to the url by the function 'format()'. The two steps of the data collection are here shown in separate functions but should be used in one were they can be compined in one for-loop. For a beginning the dataframes are saved locally as csv file. </p>